In [10]:
import numpy as np
import scipy.interpolate
import netCDF4 as nc
import os 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from helpers import visualize_h as vis_helper


In [11]:
def netcdf_to_array(file_path: str, var_name: str) -> np.ndarray:
    """
    Converts a netCDF file to a three-dimensional numpy array.

    Parameters:
        file_path (str): Path to the netCDF file.
        var_name (str): Name of the variable to extract.

    Returns:
        np.ndarray: Three-dimensional array of the specified variable.
    """
    # Open the netCDF file
    dataset = nc.Dataset(file_path, 'r')
    
    # Extract the variable data
    variable_data = dataset.variables[var_name][:]
    
    # Ensure the data is a numpy array
    array_data = np.array(variable_data)
    
    # Close the dataset
    dataset.close()
    
    return array_data

In [12]:
# Function to interpolate to target dimensions
def interpolate_to_target_dim(data, target_dim):
    m, n = data.shape[1:]  # Original dimensions
    interpolated_data = np.empty((data.shape[0], target_dim[0], target_dim[1]))  # Create an empty array to hold the results

    for i in range(data.shape[0]):  # Iterate over the time dimension
        interpolator = scipy.interpolate.RegularGridInterpolator(
            (np.linspace(0, 1, m), np.linspace(0, 1, n)), 
            data[i, :, :], 
            method='linear'
        )
        new_x = np.linspace(0, 1, target_dim[0])
        new_y = np.linspace(0, 1, target_dim[1])
        new_grid_x, new_grid_y = np.meshgrid(new_x, new_y, indexing='ij')
        points = np.stack([new_grid_x.ravel(), new_grid_y.ravel()], axis=-1)
        interpolated_slice = interpolator(points).reshape(target_dim)
        interpolated_data[i, :, :] = interpolated_slice

    return interpolated_data





Relative Paths

In [13]:
data_relative_path = os.path.join('..', 'Data')
cmip_models_path_path = os.path.join(data_relative_path,'CMIP6','Precipitation')
interpulated_cmip_path = os.path.join(data_relative_path,'InterpulatedCMIP6_400X600')


turning all CMPI6 models into interpulated matrix of dim (30,400,600)
and then save it as a npy object 

In [14]:
# # Iterate over each model file in the directory
# for model_name in os.listdir(cmip_models_path_path):
#     print(model_name)
#     model_path = os.path.join(cmip_models_path_path, model_name)
#     chirps_data_us = nc.Dataset(model_path)
#     precip_as_ndarray = netcdf_to_array(model_path, 'pr')

#     # Interpolate each time step to 400x600
#     interpolated_precip = interpolate_to_target_dim(precip_as_ndarray, (400, 600))
#     # Save the entire 3D array as a single .npy file
#     save_path = os.path.join(interpulated_cmip_path, f"{model_name[:-3]}_interpolated.npy")
#     np.save(save_path, interpolated_precip)

Showing all interpulated matrices shapes

In [22]:
# Iterate over each file in the directory
for model_name in os.listdir(interpulated_cmip_path):
    if model_name.endswith('py'):  # Only process .npy files
        file_path = os.path.join(interpulated_cmip_path, model_name)
        
        # Load the .npy file
        data = np.load(file_path)
        
        # Print the shape of the loaded data
        print(f"File: {model_name}, Shape: {data.shape}")

File: pr_Amon_ACCESS-CM2_historical_r1i1p1f1_gn_19811016-20101016_v20191108_interpolated.npy, Shape: (30, 400, 600)
File: pr_Amon_CAMS-CSM1-0_historical_r1i1p1f1_gn_19811016-20101016_v20190708_interpolated.npy, Shape: (30, 400, 600)
File: pr_Amon_CanESM5-CanOE_historical_r1i1p2f1_gn_19811016-20101016_v20190429_interpolated.npy, Shape: (30, 400, 600)
File: pr_Amon_CMCC-CM2-HR4_historical_r1i1p1f1_gn_19811016-20101016_v20200904_interpolated.npy, Shape: (30, 400, 600)
File: pr_Amon_CNRM-CM6-1-HR_historical_r1i1p1f2_gr_19811016-20101016_v20191021_interpolated.npy, Shape: (30, 400, 600)
File: pr_Amon_E3SM-1-0_historical_r1i1p1f1_gr_19811016-20101016_v20190913_interpolated.npy, Shape: (30, 400, 600)
File: pr_Amon_EC-Earth3-AerChem_historical_r1i1p1f1_gr_19811016-20101016_v20200624_interpolated.npy, Shape: (30, 400, 600)
File: pr_Amon_FGOALS-f3-L_historical_r1i1p1f1_gr_19811016-20101016_v20190927_interpolated.npy, Shape: (30, 400, 600)
File: pr_Amon_FIO-ESM-2-0_historical_r1i1p1f1_gn_19811016

# creating csv files of corr between the interpulated CMIP to the cirps 
 

In [16]:
# TODO here we need to create a DMI matrix and make the corr matrix

for model_name in os.listdir(interpulated_cmip_path):
    if model_name.endswith('py'):  # Only process .npy files
        file_path = os.path.join(interpulated_cmip_path, model_name)
        
        # Load the .npy file
        data = np.load(file_path)
        
        # Print the shape of the loaded data
        print(f"File: {model_name}, Shape: {data.shape}")